In [2]:
import numpy as np
import pandas as pd

In [3]:
latent_dim = 100
sigma = 1
sigma_p = 1
sigma_q = 1
mu = 0
rating_path = '../data/ml-latest-small/ratings.csv'

In [4]:
rating_pd = pd.read_csv(rating_path)
num_user = np.unique(rating_pd.values[:, 0]).shape[0] # 610 users
num_movie = np.unique(rating_pd.values[:, 1]).shape[0] # 9724 movies
print(num_user)
print(num_movie)
rating_pd.head()

610
9724


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
r_matrix = np.zeros((num_user, num_movie)) # 610 x 9724
p_matrix = np.random.normal(mu, sigma_p, (num_user, latent_dim)) # 610 x 100
q_matrix = np.random.normal(mu, sigma_q, (num_movie, latent_dim)) # 9724 x 100

In [6]:
movie_dic = {} 
idx = 0
for movie_id in np.unique(rating_pd.values[:, 1]): 
    movie_dic[movie_id] = idx
    idx += 1

for index, row in rating_pd.iterrows():
    r_matrix[int(row['userId'] - 1), movie_dic[row['movieId']]] = row['rating']

In [115]:
# print(r_matrix)

In [7]:
def MSE (r, p, q):
    p_qt_matrix = np.dot(p, q.T) # predicted matrix: 610x100 x 100x9724 = 610x9724
    result = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 0.5
    result = np.power(result, 2)
    result = np.multiply(I, result)
    return np.sum(result)
    
def L2_loss(sig1, sig2, matrix):
    result = np.power(matrix, 2)
    result = np.sum(result)
    constant = sig1 / float(sig2 * 2)
    return constant * result    

In [8]:
error = MSE(r_matrix, p_matrix, q_matrix) + L2_loss(sigma, sigma_q, q_matrix) + L2_loss(sigma, sigma_p, p_matrix)

In [9]:
error

6085828.129246341

In [10]:
def mat (r, p, q):
    p_qt_matrix = np.dot(p, q.T)
    result = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 0.5
    result = np.power(result, 2)
    result = np.multiply(I, result)
    return result

In [11]:
learning_rate = 0.000001

for i in range(10):
    
    a_matrix = mat(r_matrix, p_matrix, q_matrix) 
    
    error_p_d = -2*np.dot(a_matrix,np.negative(q_matrix))+0+sigma/2*sigma_p*(2*np.sum(p_matrix))
    
    error_q_d = -2*np.dot(a_matrix.T,p_matrix)+sigma/2*sigma_q*(2*np.sum(q_matrix))+0 

    p_matrix = p_matrix - learning_rate * error_p_d 
    
    q_matrix = q_matrix - learning_rate * error_q_d 
    
    error = MSE(r_matrix, p_matrix, q_matrix)
    
    print(error)

5601772.245064896
5635227.359038397
5669091.26199475
5703350.876370727
5738000.3570601065
5773032.718346814
5808439.493174442
5844210.609371553
5880334.269459093
5916796.829176258


In [34]:
learning_rate = 0.000001

for i in range(10):
    error_p_d_a = 2*np.dot(a_matrix,np.negative(q_matrix))
    error_p_d_b = sigma/2*sigma_p*(2*np.sum(p_matrix))
    
    a_matrix = mat(r_matrix, p_matrix, q_matrix) 
    
    error_p_d = error_p_d_a + error_p_d_b
    error_q_d = -2*np.dot(a_matrix.T,p_matrix) + sigma/2*sigma_q*(2*np.sum(q_matrix))

    p_matrix = p_matrix - learning_rate * error_p_d 
    q_matrix = q_matrix - learning_rate * error_q_d 
    
    print(error_p_d_a)
    print(":")
    print(error_p_d_b)
    
    error = MSE(r_matrix, p_matrix, q_matrix)
    print(error)

[[ 2.11895114e+03  3.43347842e+02  1.61910341e+03 ...  1.08136787e+03
  -6.47768677e+03 -5.87247241e+03]
 [-6.79573971e+02  3.03295572e+02  2.72603745e+02 ... -1.85067745e+02
  -1.39783375e+02 -9.70823518e+01]
 [-7.36943600e+02 -2.52737271e+01  1.26496244e+02 ...  3.37767054e+01
  -4.04156197e+02  9.61660173e+02]
 ...
 [-2.09851003e+03 -3.47123427e+03  2.09231505e+04 ...  8.02496091e+02
  -1.31926680e+03  1.18740518e+04]
 [-1.59516825e+03 -3.71621128e+02  1.67354372e+02 ...  4.55346403e+02
   2.66401107e+02  3.69383276e+02]
 [ 9.26387473e+03  3.90726407e+04  2.83644473e+04 ... -3.47170036e+04
  -5.47304961e+04  5.27160274e+04]]
:
-6.322150013747859
7352744.448759901
[[ 2.13000076e+03  4.15675665e+02  1.63646616e+03 ...  1.02270877e+03
  -6.36070854e+03 -5.84029509e+03]
 [-6.77666676e+02  3.03379732e+02  2.76992204e+02 ... -1.64910366e+02
  -1.43111006e+02 -9.65839952e+01]
 [-7.36020691e+02 -2.35356483e+01  1.25678890e+02 ...  3.42720370e+01
  -4.12264414e+02  9.68097143e+02]
 ...
 [-2.